# Lot compounding

This notebook looks at methods on compounding lots; futures and options contracts that could be traded only in specific lots. Since these cannot be compounded in a fine grained manner, let us start with some simple methods.

Assumptions
------------

 1. Profit and loss is expressed in points per contract
 2. Capital required per contract is fairly stable over time. This is important in case you are **selling options**.
 3. Commissions and slippage is included in the profit and loss
 
 

## Simple Method

In the simple method, we allocate capital per lot based on capital required per lot and add maximum expected loss for a single lot. Maximum expected loss is the maximum loss per trade multiplied by the number of expected consecutive losing trade. We assume we could **estimate maximum loss per trade reasonably well.**


Example
-------
    Lot size = 100
    Capital required per lot = 50000
    Maximum loss in points per trade = 50
    Expected consecutive losses = 10
    Expected maximum loss = 50*10*100 = 50000
    Total capital to be allocated = 50000+50000 = 100000
    

So, lots would be increased if we earn necessary capital per lot and we would keep on trading till we exhaust our buffer. In this case, we would increase the lot size if we earn 50000 and keep on trading till our total capital falls to 50000.

> Number of lots to trade = **int((total capital - expected maximum loss)/capital per lot)**


In [6]:
import pandas as pd
import numpy as np
from typing import Union, List, Tuple

In [16]:
def lot_compounding(pnl:Union[pd.Series, List, Tuple], initial_capital:float=100000,
                   capital_per_lot:float=50000, buffer:float=50000, lot_size:float=100)->pd.DataFrame:
    """
    Compounding based on loss
    pnl
        profit or loss in points per contract
        points should not be multiplied by lot size
    initial_capital
        initial capital invested includes both capital per lot and buffer
    capital_per_lot
        capital required per lot
    buffer
        buffer for drawdown for a single lot
    lot_size
        lot size
    """
    capital = initial_capital
    capital_arr = []
    lots_arr = []
    for p in pnl:
        lots = int((capital-buffer)/capital_per_lot)
        if lots == 0:
            if capital < capital_per_lot:
                lots = 1
        if lots > 0:
            profit = p*lot_size*lots
            capital += profit
        capital_arr.append(capital)
        lots_arr.append(lots)
        index = [pnl.index if type(pnl)==pd.Series else np.arange(len(pnl))]
    return pd.DataFrame({
        'capital': capital_arr,
        'lots': lots_arr
    }, index=index)

            

## Let us try a few simple cases

In [18]:
pnl = [100] * 6
lot_compounding(pnl)

,capital,lots
0,110000,1
1,120000,1
2,130000,1
3,140000,1
4,150000,1
5,170000,2
